## Install and Import Package
Installing the **ultralytics** library
**Ultralytics** is a library that provides tools for working with the YOLOv8 model. YOLOv8 is an advanced model for object detection, image segmentation, and image classification

In [1]:
!pip install ultralytics


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Downloading the Required Libraries


In [2]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob 
import random
import os

## Clone Git Repo
Copying data from a repository GitHub

In [3]:
#!git clone https://github.com/ThatCoderMan/Hay_test_task.git
#!cp -rf ./Hay_test_task/* ./
#!rm -r Hay_test_task/

## Download the Dataset
Downloading prepared masks from the site [roboflow](https://app.roboflow.com/ds)

In [4]:
# os.makedirs('datasets', exist_ok=True)
# %cd datasets
# !curl -L "https://app.roboflow.com/ds/mzocbskx4m?key=Eb2lIuRA0D" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# %cd ..

Downloading the original video from Google drive

In [5]:
# %cd data/videos
# !wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1_lWgETFS_sxk9tzgtteI5PZEeGzTdIXq&export=download&authuser=0&confirm=t&uuid=734bc640-a87a-48f7-b451-3d7d7946b78d&at=APZUnTW9SicfuV7bc74bHdfxhNjp:1698261775232' -O video.mp4
# %cd ../..
# %pwd

## Visualize Images from the Dataset
Checking formats on prepared data

In [6]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [7]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))
                
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

# Function to plot images with the bounding boxes.

In [8]:

def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    
    all_images.sort()

    num_images = len(all_images)
    
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

# Visualize a few training images.

In [9]:

#plot(
    # image_paths='../../datasets/train/images/',
    # label_paths='../../datasets/train/labels/',
    # num_samples=4,
# )

## Dataset YAML File
Creating a File YAML for _Training_ and _Validation_ Files

In [10]:
# %%writefile hay.yaml
# path: ''
# train: 'train/images'
# val: 'valid/images'
#
# names:
#   0: 'hay'

## YOLOv8 Large Training

In [11]:
%pwd

'C:\\Users\\rokeb\\PycharmProjects\\Hay_test_task\\notebooks'

In [12]:
import os
import torch
import torchvision
print(f'{torch.cuda.is_available()=}')
print(f'{torch.__version__=}')
print(f'{torchvision.__version__=}')
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

from ultralytics import YOLO

model = YOLO('yolov8l.pt')
result = model.train(data='hay.yaml', epochs=100, batch=16, device=[0])

# EPOCHS = 20
# !yolo task=detect mode=train model=yolov8l.pt epochs={EPOCHS} batch=16 name=hay_model data=hay.yaml device=0

torch.cuda.is_available()=True
torch.__version__='2.1.0+cu121'
torchvision.__version__='0.16.0+cu121'


New https://pypi.org/project/ultralytics/8.0.221 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208  Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov8l.pt, data=hay.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train32, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnos

10## Inference on Validation Images1
Video-to-video tracking command

In [13]:
# !yolo track \
# model=runs/detect/hay_model/weights/best.pt \
# source=data/videos/Seno1.mp4 \
# name=result \
# hide_labels=True \
# conf=0.1 iou=0.75

Command to localize objects from photo to photo

In [14]:
# !yolo task=detect \
# mode=predict \
# model=runs/detect/hay_model/weights/best.pt \
# source=data/videos/original_video \
# name=result \
# hide_labels=True

Convert Photos to Videos

In [15]:
# !ffmpeg -f image2 -r 24 -i ./runs/detect/result/frame_%04d.jpg  result.mp4 

## Visualize Validation Results
Display of processed data

In [16]:
# Plot and visualize images in a 2x2 grid.
def visualize(result_dir, num_samples=4):
    """
    Function accepts a list of images and plots
    them in a 2x2 grid.
    """
    plt.figure(figsize=(20, 12))
    image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names):
        image = plt.imread(image_name)
        plt.subplot(2, 2, i+1)
        plt.imshow(image)
        plt.axis('off')
        if i == num_samples-1:
            break
    plt.tight_layout()
    plt.show()

In [17]:
visualize('runs/detect/hay_model10/')

<Figure size 2000x1200 with 0 Axes>